In [1]:
import os
import sys
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from tqdm import tqdm

root_dir = "../"
data_dir = os.path.join(root_dir, "data")
sys.path.append(root_dir)

from dataset import HemorrhageDataset


In [2]:
train_image_dataset = HemorrhageDataset(
    data_dir=data_dir, op="train", augment=False, weak_supervision=False
)
train_image_dataloader = DataLoader(
    train_image_dataset, batch_size=1, num_workers=4, shuffle=True
)

denorm = lambda x, mean, std: x * std[:, None, None] + mean[:, None, None]
MEAN = train_image_dataset.MEAN
STD = train_image_dataset.STD

neg_count = 0
for data in tqdm(train_image_dataloader):
    image, label = data

    image = image.squeeze()
    label = label.squeeze()

    if label == 0:
        denorm_image = denorm(image, MEAN, STD)
        _, ax = plt.subplots()
        ax.imshow(denorm_image.permute(1, 2, 0))
        ax.axis("off")
        plt.savefig(f"neg_{neg_count}.jpg", bbox_inches="tight")
        plt.savefig(f"neg_{neg_count}.pdf", bbox_inches="tight")
        plt.close()
        neg_count += 1
    
    if neg_count == 10:
        break


  0%|          | 9/601930 [00:04<77:13:35,  2.17it/s] 
